In [7]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import copy
import pandas as pd
import time

In [8]:
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score

In [9]:
from subspace_clustering_helper_funcs import *

- https://scikit-learn.org/stable/modules/classes.html#module-sklearn.manifold

## Loading in the data

In [10]:
# remove pID 101 because it doesn't exist
# remove pID 131 because it  doesnt have enough user defined gestures
# each participant has 100 experimenter defined files and 50 user defined files
# 10 experimenter defined gestures and 5 user defined gestures

file_types = ["IMU_extract", "movavg_files"]
expt_types = ["experimenter-defined"]

#remove participant 131 because they are missing gestures 
pIDs_impaired = ['P102','P103','P104','P105','P106','P107','P108','P109','P110','P111',
       'P112','P114','P115','P116','P118','P119','P121','P122','P123','P124','P125',
       'P126','P127','P128', 'P132']
# remove participants P001 and P003 because they dont have duplicate or open gestures
pIDs_unimpaired = ['P004','P005','P006','P008','P010','P011']

pIDs_both = pIDs_impaired + pIDs_unimpaired

In [11]:
## Pickle is theoretically faster for Python...

# Kai's laptop
data_path = "C:\\Users\\kdmen\\Desktop\\Research\\Data\\$M\\PCA_40D\\"
# BRC Desktop
#data_path = "D:\\Kai_MetaGestureClustering_24\\saved_datasets\\"

print("Loading")
start_time = time.time()

metadata_cols = ['Participant', 'Gesture_ID', 'Gesture_Num']

PCA_df = pd.read_pickle(data_path+'PCA_ms_IMUEMG_df.pkl')
metadata_cols_df = pd.read_pickle('C:\\Users\\kdmen\\Desktop\\Research\\Data\\$M\\metadata_cols_df.pkl')

# Dropping the metadata when we read it in!
training_u_df = pd.read_pickle(data_path+'training_u_df.pkl').drop(metadata_cols, axis=1)
test_users_df = pd.read_pickle(data_path+'test_users_df.pkl').drop(metadata_cols, axis=1)

training_g_df = pd.read_pickle(data_path+'training_g_df.pkl').drop(metadata_cols, axis=1)
test_fullgestures_df = pd.read_pickle(data_path+'test_fullgestures_df.pkl').drop(metadata_cols, axis=1)

end_time = time.time()
print(f"Completed in {end_time - start_time}s")

Loading
Completed in 18.70682168006897s


In [12]:
print(training_u_df.shape)
training_u_df.head()

(327168, 40)


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-0.027903,0.001411,-0.019509,0.013428,-0.019699,0.027333,-0.031254,-0.022910,0.066484,0.108729,...,-0.019453,0.062983,-0.025869,0.014303,-0.013387,-0.037645,-0.186270,-0.046251,-0.104630,-0.002939
1,-0.038982,0.006470,-0.000111,0.010904,-0.015323,0.031336,-0.007901,-0.027368,0.060370,0.074712,...,0.041438,0.035053,-0.056843,-0.008895,-0.022542,-0.022563,-0.160826,-0.048161,-0.073771,0.043268
2,-0.116782,0.003824,0.011550,-0.014612,-0.093325,0.081718,-0.013155,-0.046150,0.036385,0.052746,...,-0.014298,0.072109,-0.026536,-0.034365,0.018695,-0.011940,-0.160580,-0.041831,-0.109653,0.027043
3,-0.030245,-0.017409,0.022540,-0.048905,-0.029129,0.090026,-0.024645,-0.064307,0.074589,0.053055,...,-0.010992,0.059990,-0.097073,-0.056870,-0.001038,-0.008015,-0.165858,-0.049424,-0.108671,0.069886
4,-0.112950,0.026262,0.004837,-0.063254,-0.108892,0.198729,-0.010583,-0.124893,0.114817,0.038628,...,0.035735,0.050880,-0.093678,-0.131263,0.018035,0.056185,-0.157963,-0.041911,-0.145308,0.063311


In [13]:
print(test_users_df.shape)
test_users_df.head()

(99584, 40)


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
11520,0.068817,0.042105,-0.044102,0.053256,0.080938,0.106052,-0.022074,0.036360,-0.069783,-0.009365,...,0.063422,0.081947,0.101682,-0.190886,-0.128849,-0.133675,-0.141095,-0.068262,-0.032521,0.018558
11521,0.074061,0.047283,-0.042408,0.054238,0.074058,0.102463,-0.015359,0.038657,-0.073921,-0.017769,...,0.072693,0.087035,0.107107,-0.189667,-0.127273,-0.140950,-0.143033,-0.066269,-0.033821,0.018198
11522,0.079972,0.043184,-0.055275,0.046477,0.079097,0.100367,-0.017235,0.037957,-0.077119,-0.026219,...,0.071150,0.081851,0.103781,-0.186400,-0.119714,-0.137523,-0.143988,-0.064562,-0.036610,0.016388
11523,0.066582,0.035923,-0.040610,0.045127,0.083319,0.103084,-0.026479,0.037962,-0.074883,-0.020225,...,0.065488,0.077696,0.100158,-0.195122,-0.128462,-0.133530,-0.147047,-0.064546,-0.032468,0.015712
11524,0.070243,0.048672,-0.050055,0.045911,0.075328,0.102190,-0.026261,0.038253,-0.068708,-0.025581,...,0.064663,0.082686,0.104846,-0.185309,-0.126085,-0.138847,-0.147857,-0.067871,-0.031347,0.019686


In [14]:
print(training_g_df.shape)
training_g_df.head()

(383424, 40)


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-0.027903,0.001411,-0.019509,0.013428,-0.019699,0.027333,-0.031254,-0.022910,0.066484,0.108729,...,-0.019453,0.062983,-0.025869,0.014303,-0.013387,-0.037645,-0.186270,-0.046251,-0.104630,-0.002939
1,-0.038982,0.006470,-0.000111,0.010904,-0.015323,0.031336,-0.007901,-0.027368,0.060370,0.074712,...,0.041438,0.035053,-0.056843,-0.008895,-0.022542,-0.022563,-0.160826,-0.048161,-0.073771,0.043268
2,-0.116782,0.003824,0.011550,-0.014612,-0.093325,0.081718,-0.013155,-0.046150,0.036385,0.052746,...,-0.014298,0.072109,-0.026536,-0.034365,0.018695,-0.011940,-0.160580,-0.041831,-0.109653,0.027043
3,-0.030245,-0.017409,0.022540,-0.048905,-0.029129,0.090026,-0.024645,-0.064307,0.074589,0.053055,...,-0.010992,0.059990,-0.097073,-0.056870,-0.001038,-0.008015,-0.165858,-0.049424,-0.108671,0.069886
4,-0.112950,0.026262,0.004837,-0.063254,-0.108892,0.198729,-0.010583,-0.124893,0.114817,0.038628,...,0.035735,0.050880,-0.093678,-0.131263,0.018035,0.056185,-0.157963,-0.041911,-0.145308,0.063311


In [15]:
print(test_fullgestures_df.shape)
test_fullgestures_df.head()

(43328, 40)


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
11520,0.068817,0.042105,-0.044102,0.053256,0.080938,0.106052,-0.022074,0.036360,-0.069783,-0.009365,...,0.063422,0.081947,0.101682,-0.190886,-0.128849,-0.133675,-0.141095,-0.068262,-0.032521,0.018558
11521,0.074061,0.047283,-0.042408,0.054238,0.074058,0.102463,-0.015359,0.038657,-0.073921,-0.017769,...,0.072693,0.087035,0.107107,-0.189667,-0.127273,-0.140950,-0.143033,-0.066269,-0.033821,0.018198
11522,0.079972,0.043184,-0.055275,0.046477,0.079097,0.100367,-0.017235,0.037957,-0.077119,-0.026219,...,0.071150,0.081851,0.103781,-0.186400,-0.119714,-0.137523,-0.143988,-0.064562,-0.036610,0.016388
11523,0.066582,0.035923,-0.040610,0.045127,0.083319,0.103084,-0.026479,0.037962,-0.074883,-0.020225,...,0.065488,0.077696,0.100158,-0.195122,-0.128462,-0.133530,-0.147047,-0.064546,-0.032468,0.015712
11524,0.070243,0.048672,-0.050055,0.045911,0.075328,0.102190,-0.026261,0.038253,-0.068708,-0.025581,...,0.064663,0.082686,0.104846,-0.185309,-0.126085,-0.138847,-0.147857,-0.067871,-0.031347,0.019686


Let's actually re-arrange to make them 3D matrices...

In [16]:
num_rows_per_gesture = 64 # From the interp
num_gestures = len(PCA_df) // num_rows_per_gesture
num_features = PCA_df.shape[1]

# Ensure the data can be evenly divided into gestures
assert len(PCA_df) % num_rows_per_gesture == 0, "The total number of rows is not a multiple of the number of rows per gesture."

# Convert DataFrame to NumPy array
data_np = PCA_df.to_numpy()
# Reshape into (batch_dim, time_step, n_features) AKA (n_gestures, n_rows_per_gesture, n_columns)
PCA_np = data_np.reshape(num_gestures, num_rows_per_gesture, num_features)

#PCA_np = PCA_df.to_numpy()
flattened_PCA = PCA_np.reshape(num_gestures, -1)

Now proceed with the clustering

In [17]:
# Maximum number of components
max_clusters = 20 # Original dataset size

# Create a combined range: 2-20
num_clusters_range = list(range(2, max_clusters+1))

In [18]:
PCA_np.shape

(6668, 64, 40)

In [19]:
flattened_PCA.shape

(6668, 2560)

## Hierarchy / Dendrogram
Now, let's investigate a more informed choice, based on measured statistical heterogenity. Could look at Shannon or KL divergence (heatmap), but I'll just look at a dendogram for now.

In [20]:
data_df.head()

NameError: name 'data_df' is not defined

In [ ]:
data_df['Gesture_ID'].unique()

In [ ]:
len(data_df['Gesture_ID'].unique())

In [ ]:
len(data_df['Gesture_Num'].unique())

In [ ]:
len(data_df['Participant'].unique())

In [ ]:
subset_df = data_df[data_df['Participant'] == 'P102']
print(subset_df.shape)
subset_df.head()

In [ ]:
for p in data_df['Participant'].unique():
    subset_df = data_df[data_df['Participant'] == p]
    print(subset_df.shape)

In [ ]:
assert(1==0)

In [ ]:
# This code is not ready to run yet...
## Does not handle data assymetries (eg different dataset sizes between clients)

n_clients = len(data_df['Participant'].unique())
n_trials = 8
# Flatten data for easy computation
flattened_data = np.vstack(data_clients2)
# Reshape data to separate trials for each client
reshaped_data = flattened_data.reshape(n_clients, n_trials, -1)
# Calculate pairwise distances between clients' trial data
inter_subject_distances = np.zeros((n_clients, n_clients))
for i in range(n_clients):
    for j in range(i + 1, n_clients):
        distance = np.mean(np.linalg.norm(reshaped_data[i] - reshaped_data[j], axis=1))
        inter_subject_distances[i, j] = distance
        inter_subject_distances[j, i] = distance
# Convert inter-subject distances to a condensed distance matrix
condensed_distances = squareform(inter_subject_distances)

# Perform hierarchical clustering
linkage_matrix_single = linkage(condensed_distances, method='single')
# Plot dendrogram
plt.figure(figsize=(10, 6))
dendrogram(linkage_matrix_single, labels=np.arange(n_clients))
plt.title("Hierarchical Clustering Dendrogram")
plt.xlabel("Client Index")
plt.ylabel("Distance")
plt.show()